In [1]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

In [2]:
# Load pre-trained BERT tokenizer and model fine-tuned on SQuAD
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

C:\Users\mshos\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mshos\.cache\huggingface\hub\models--bert-large-uncased-whole-word-masking-finetuned-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
# Define the question and context (passage)
question = "Where did the cat sit?"
context = "The cat sat on the mat near the window. It looked very comfortable."

In [4]:
# Encode the question and context together
inputs = tokenizer(question, context, return_tensors="pt")


In [5]:
# Run the model to get start and end logits for the answer span
with torch.no_grad():
    outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits


In [6]:
# Get the most probable start and end positions
start_index = torch.argmax(start_logits)
end_index = torch.argmax(end_logits)

In [7]:
# Decode the answer from the input ids
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index : end_index + 1])
)

In [8]:

print("Question:", question)
print("Answer:", answer)

Question: Where did the cat sit?
Answer: near the window


In [11]:
start_logits

tensor([[-5.4289, -5.9347, -7.3897, -7.4089, -6.7239, -7.3540, -8.8766, -5.4288,
          1.7281, -1.2975,  0.3599,  4.1286,  3.1749,  2.4569,  5.1155,  0.7299,
          2.3086, -5.3806, -2.9994, -5.5147, -5.4894, -3.9346, -5.4294, -5.4286]])

In [12]:
end_logits

tensor([[-0.1619, -6.4591, -6.8918, -7.6206, -6.0679, -6.5172, -7.4069, -0.1615,
         -5.1359, -3.0821, -2.9711, -2.3550, -2.7320,  2.4637, -0.8783, -3.7991,
          7.0502,  3.5367, -4.9805, -6.5219, -6.6309, -1.2670, -0.1620, -0.1621]])